In [5]:
import nbimporter

In [6]:
import utils
import blockchain

### PARSE RESOURCES

In [7]:
binary_interfaces = utils.load_json('resources/binary.json')

In [8]:
settings = utils.load_json('resources/settings.json')

### CONNECT WITH A BLOCKCHAIN GATEWAY

In [9]:
web3_instance = blockchain.connect(settings)

CONNECTION ESTABLISHED!


### SERIALIZE THE DEVICE MANAGER

In [10]:
device_manager = blockchain.contract(
    binary_interfaces['devicemanager'],
    web3_instance, settings
)

### CREATE A NEW DEVICE CONTRACT

In [11]:
identifier = 'another-identifier'

In [12]:
name = 'foobar'

In [13]:
device_manager.write({
    'func': 'create_device',
    'args': [identifier, name]
})

AttributeDict({'transactionHash': HexBytes('0x81b7611692a56224ace14840d775f730421d93ababe8144df9124a055ff33290'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xd95d13d6acbb2e3787e8521c69c9550336ac0a489f747d373192347b304c6787'),
 'blockNumber': 5,
 'from': '0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e',
 'to': '0xCfEB869F69431e42cdB54A4F4f105C19C080A601',
 'gasUsed': 449578,
 'cumulativeGasUsed': 449578,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

### FETCH YOUR DEVICE COLLECTION (IDENTIFIERS)

In [14]:
collection =  device_manager.read({
    'func': 'fetch_collection',
    'args': settings['keys']['public']
})

In [15]:
collection

['another-identifier']

### FETCH THE CREATED DEVICE CONTRACT'S ADDRESS BY USING THE IDENTIFIER

In [16]:
device_address =  device_manager.read({
    'func': 'fetch_device',
    'args': identifier
})

In [17]:
device_address

'0x79183957Be84C0F4dA451E534d5bA5BA3FB9c696'

### SERIALIZE THE DEVICE CONTRACT

In [18]:
device_contract = blockchain.contract({
    'address': device_address,
    'abi': binary_interfaces['device']['abi']
}, web3_instance, settings)

### FETCH THE DEVICE CONTRACT'S DETAILS

In [19]:
details = device_contract.read('details')

In [20]:
print('OWNER:\t\t', details[0])
print('IDENTIFIER:\t', details[1])
print('NAME:\t\t', details[2])
print('CREATED:\t', details[3])
print('ACTIVE:\t\t', details[4])

OWNER:		 0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e
IDENTIFIER:	 another-identifier
NAME:		 foobar
CREATED:	 5
ACTIVE:		 False


### CHANGE THE DEVICE CONTRACT ACTIVE STATUS

In [22]:
device_contract.write('toggle_active')

AttributeDict({'transactionHash': HexBytes('0xc6222707d7cd5810682461389ad904749d5168cd6618093fe20feec1909da74d'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x6a2521b34023ee60f9999e84ef36f37ab75de8b576707c6a34016e05235bf094'),
 'blockNumber': 7,
 'from': '0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e',
 'to': '0x79183957Be84C0F4dA451E534d5bA5BA3FB9c696',
 'gasUsed': 13944,
 'cumulativeGasUsed': 13944,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')

### CHECK ACTIVE STATUS AGAIN

In [23]:
status = device_contract.read('active')

In [19]:
status

True